# Bosonic.py

In [14]:
import numpy as np
from scipy.special import binom 
from scipy.sparse import dok_matrix, linalg
from scipy import linalg as linalg_d
from joblib import Memory
import random
import plotly.graph_objects as go
from joblib import Parallel, delayed

location = './cachedir'
memory = Memory(location, verbose=0)

In [2]:
class fixed_basis:

    # Convierte a un enterno n a su escritura en base b
    def int_to_tuple(self, n, b, digits = None):
        rep = np.base_repr(n, b)
        rep_int = [int(x,b) for x in rep] 
        if digits is not None:
            zeros = [0 for i in range(0,digits-len(rep))]
            return zeros + rep_int
        else:
            return rep_int
    
    # Revierte la transformacion anterior
    def tuple_to_int(self, t):
        b = self.d-1
        l = len(t)
        s = [t[k]*b**(l-k-1) for k in range(0,l)]
        return sum(s)

    # Convierte el vector en su representacion
    def vect_to_repr(self, vect):
        for i, k in enumerate(vect):
            if k == 1. or k == 1:
                break
        return self.base[i,:]
            
    def rep_to_vect(self, rep):
        rep = list(rep)
        for i, r in [(j, self.base[j,:]) for j in range(0,self.size)]:
            if list(r) == rep:
                return self.canonicals[:,i]
        else:
            None
    
    def rep_to_index(self, rep):
        rep = list(rep)
        for i, r in [(j, self.base[j,:]) for j in range(0,self.size)]:
            if list(r) == rep:
                return i
        else:
            None

    @staticmethod
    def rep_to_exi(rep):
        r = []
        for i, k in enumerate(rep):
            r += [i for x in range(0,k)]
        return r

    # Crea base de M particulas en D estados (repr y base canonica)
    def create_basis(self, m, d):
        length = int(binom(d+m-1,d-1))
        base = []
        index = 0
        for x in range(0,(d+1)**max(d,m)):
            x = self.int_to_tuple(x,m+1,d)
            if sum(x) == m and len(x) == d:
                base.append(x)
                index += 1
        base = np.array(base)
        # Asignamos a cada uno de ellos un canónico
        x = [1.0 for j in range(0,length)]
        canonicals = np.diag(x)
        return base, canonicals
    
    def __init__(self, m, d):
        self.m = m
        self.d = d
        self.size = int(binom(d+m-1,d-1))
        (self.base, self.canonicals) = self.create_basis(m, d)

# Matrices de aniquilación y creación endomórficas. Estan fuera de la clase para poder ser cacheadas        
@memory.cache    
def bdb(basis, i, j):
    mat = dok_matrix((basis.size, basis.size), dtype=np.float32)
    if i != j:
        for k, v in enumerate(basis.base):
            if v[j] != 0:
                dest = list(v.copy())
                dest[j] -= 1
                dest[i] += 1
                tar = basis.rep_to_index(dest)
                mat[tar, k] = np.sqrt(v[i]+1)*np.sqrt(v[j])
    else:
        for k, v in enumerate(basis.base):
            if v[j] != 0:
                mat[k, k] = v[i] 
    return mat

@memory.cache    
def bbd(basis, i, j):
    mat = dok_matrix((basis.size, basis.size), dtype=np.float32)
    if i != j:
        for k, v in enumerate(basis.base):
            if v[i] != 0:
                dest = list(v.copy())
                dest[i] -= 1
                dest[j] += 1
                tar = basis.rep_to_index(dest)
                mat[tar, k] = np.sqrt(v[j]+1)*np.sqrt(v[i])
    else:
        for k, v in enumerate(basis.base):
            mat[k, k] = v[i]+1
    return mat

# Matrices de aniquilación y creación.Toman la base de origen y destino (basis_o, basis_d) resp
@memory.cache   
def b(basis_o, basis_d, i):
    mat = dok_matrix((basis_d.size, basis_o.size), dtype=np.float32)
    for k, v in enumerate(basis_o.base):
        if v[i] != 0:
            dest = list(v.copy())
            dest[i] -= 1   
            tar = basis_d.rep_to_index(dest)
            mat[tar, k] = np.sqrt(v[i])
    return mat


@memory.cache   
def bd(basis_o, basis_d, i):
    mat = dok_matrix((basis_d.size, basis_o.size), dtype=np.float32)
    for k, v in enumerate(basis_o.base):
        dest = list(v.copy())
        dest[i] += 1   
        tar = basis_d.rep_to_index(dest)
        mat[tar, k] = np.sqrt(v[i]+1)
    return mat

# Devuelve la matriz gamma asociada a la descomposición (M,N-M) del vector
def gamma(basis, m, vect, m_basis = None, nm_basis = None):
    d = basis.d
    if not m_basis or not nm_basis:
        m_basis = fixed_basis(m, d)
        nm_basis = fixed_basis(basis.m-m,d)
    mat = dok_matrix((m_basis.size, nm_basis.size), dtype=np.float32)
    for i, v in enumerate(m_basis.base):
        for j, w in enumerate(nm_basis.base):
            targ = v+w
            index = basis.rep_to_index(targ)
            if index == None:
                continue
            coef = vect[index]
            if coef != 0:      
                aux = lambda x: np.prod(np.reciprocal(np.sqrt([np.math.factorial(o) for o in x])))
                aux_inv = lambda x: np.prod(np.sqrt([np.math.factorial(o) for o in x]))
                coef = coef * aux(v) * aux(w) * aux_inv(targ)
            mat[i,j] = coef
    return mat

# Devuelve la matriz rho M asociada al vector
def rho_m(basis, m, vect, m_basis = None, nm_basis = None):
    g = gamma(basis, m, vect, m_basis, nm_basis)
    return np.dot(g,np.transpose(g))

# Devuelve la matriz rho 1 asociada al vector
def rho_1(basis, vect):
    d = basis.d
    mat = dok_matrix((d, d), dtype=np.float32)
    for i in range(0, d):
        for j in range(0, d):
            v = bdb(basis,j,i)*vect
            mat[i,j] = np.inner(vect,v)
    return mat
      
# Devuelve la matriz rho 2 asociada al vector
def rho_2(basis, vect, ml_basis = None, mll_basis = None, t_basis = None):
    d = basis.d
    # Creo las bases si no están dadad
    if ml_basis == None or mll_basis == None or t_basis == None:
        ml_basis = fixed_basis(m-1,d)
        mll_basis = fixed_basis(m-2,d)
        t_basis = fixed_basis(2,d)
    mat = dok_matrix((t_basis.size, t_basis.size), dtype=np.float32)
    for i, v in enumerate(t_basis.base):
        for j, w in enumerate(t_basis.base):
            # Creacion de los a
            i_set = t_basis.rep_to_exi(v)
            b_m = np.dot(b(ml_basis, mll_basis, i_set[1]),b(basis, ml_basis, i_set[0]))
            
            # Creacion de los ad
            i_set = t_basis.rep_to_exi(w)
            bd_m = np.dot(bd(ml_basis, basis, i_set[1]),bd(mll_basis, ml_basis, i_set[0]))    

            # Mult de b's y filleo de mat
            coef = np.inner(vect,np.dot(bd_m,b_m)*vect)
            aux = lambda x: np.prod(np.reciprocal(np.sqrt([np.math.factorial(o) for o in x])))
            mat[i,j] = coef * aux(v) * aux(w)
    return mat    

In [5]:
[2 for x in range(0,0)]

[]

In [ ]:
# Prueba de las matrices gamma (TODO normalizacion)
v = a.rep_to_vect([0, 1, 1, 2])+a.rep_to_vect([1, 1, 2, 0])
print(v)
gamma(a, 2, v)
v = np.array([random.random() for a in range(0,a.size)])
v = v / np.linalg.norm(v)
print(v)
gamma(a, 2, v)

[0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0.29033901 0.13673684 0.08792772 0.13363623 0.08228274 0.2349543
 0.15559778 0.15495642 0.00073171 0.15029627 0.00290446 0.283889
 0.09512864 0.20174247 0.08405987 0.02414921 0.01972325 0.11700251
 0.28425527 0.1501779  0.04730217 0.1910561  0.29681086 0.27637613
 0.12939154 0.07349414 0.16606558 0.11792164 0.27328547 0.04051742
 0.14844461 0.24804708 0.18761501 0.14194226 0.03336991]


<10x10 sparse matrix of type '<class 'numpy.float32'>'
	with 100 stored elements in Dictionary Of Keys format>

In [63]:
# Prueba de funcionamiento de las matrices bdb y bbd
v = [1, 1, 0, 2]
vect = a.rep_to_vect(v)
res = bbd(a,0,1) * vect
if np.linalg.norm(res) != 0:
    res = res / np.linalg.norm(res)
    print(a.vect_to_repr(res))
else:
    res = 0
    print(0)


[0 2 0 2]


In [69]:
# Prueba de funcionamiento de las matrices b y bd

l = fixed_basis(3, 3)
m = fixed_basis(2, 3)

vect = l.rep_to_vect([0,1,2])
res = b(l,m,2) * vect
if np.linalg.norm(res) != 0:
    print(res)
    res = res / np.linalg.norm(res)
    print(m.vect_to_repr(res))
    
vect = m.rep_to_vect([0,0,2])
res = bd(m,l,2) * vect
if np.linalg.norm(res) != 0:
    print(res)
    res = res / np.linalg.norm(res)
    print(l.vect_to_repr(res))

[0.         1.41421354 0.         0.         0.         0.        ]
[0 1 1]
[1.73205078 0.         0.         0.         0.         0.
 0.         0.         0.         0.        ]
[0 0 3]


## Pairing

Construimos el espacio para n particulas en d estados. El hamiltoniano de pairing posee energias equiespaciadas con degeneración doble (asociada a cada estado y su inverso temporal), así como un término de interacción. Por tal motivo, crearemos un espacio con d = 2d.

In [51]:
m = 6
d = 3
# Creo las bases para no tener que recrearlas luego
basis = fixed_basis(m, 2*d)
basis_m1 = fixed_basis(m-1, 2*d)
basis_m2 = fixed_basis(m-2, 2*d)
print(basis.base)
print(basis.size)



[[0 0 0 0 0 6]
 [0 0 0 0 1 5]
 [0 0 0 0 2 4]
 ...
 [5 0 1 0 0 0]
 [5 1 0 0 0 0]
 [6 0 0 0 0 0]]
462


In [52]:
# Parametros hamiltoniano
e = 1
eps = 0.0001
e0 = np.zeros(2*d)
eigenspace_tol = 0.0001
for k in range(0, d):
    r = random.random() * eps
    e0[2*k] = k*e+r
    e0[2*k+1] = k*e+r

def hamiltonian(g, basis, basis_m1, basis_m2):
    # Construccion de H
    h0 = sum([e0[k]*bdb(basis,k,k) for k in range(0,2*d)])
    hi = dok_matrix((basis.size, basis.size), dtype=np.float32)
    for k in range(0,d):
        for kb in range(0,d):
            hi += -1*g*bd(basis_m1, basis, 2*k)*bd(basis_m2, basis_m1, 2*k+1)*b(basis_m1, basis_m2, 2*kb+1)*b(basis, basis_m1, 2*kb)

    return h0+hi

def solve(g, m, last_step = None):
    h = hamiltonian(g, basis, basis_m1, basis_m2)
    sol = linalg.eigsh(h,which='SA',k=19)
    if type(last_step) != type(None):
        # Seleccionamos todos los autovects que difieren sus autovalores menos que tol (mismo autoespacio)
        # y tomamos la proyección en el autoespacio de la solución del paso anterior (last_step)
        eig = sol[0].real
        eigv = sol[1]
        cand = [eigv[:,i].real  for (i, x) in enumerate(eig) if abs(x-min(eig)) < eigenspace_tol]
        cand_norm = [x/np.linalg.norm(x) for x in cand]
        fund = np.zeros(len(cand[0]))
        for x in cand_norm:
            fund += np.dot(last_step,x) * x
        #print(np.dot(fund,last_step))
    else:
        argmin = np.argmin(sol[0].real)
        fund = sol[1][:,argmin]
    fund = fund.real / np.linalg.norm(fund)
    return fund

In [ ]:
# Rutina de resolución
m = 2
m_basis = fixed_basis(m, 2*d)
nm_basis = fixed_basis(basis.m-m, 2*d)

num = 50
g_range = np.linspace(0.01,5,num)
rho_range= {}

def compute_g(g):
    print(g)
    fund = solve(g, m)
    rho = rho_m(basis, m, fund, m_basis, nm_basis).todense()
    r = np.sort(linalg_d.eigvals(rho).real)
    rho_range[g] = r

Parallel(n_jobs=8, require='sharedmem')(
    delayed(compute_g)(g) for g in g_range);

0.01
0.11183673469387755
0.2136734693877551
0.31551020408163266
0.4173469387755102
0.5191836734693878
0.6210204081632653
0.7228571428571429
0.8246938775510204
0.926530612244898
1.0283673469387755
1.130204081632653
1.2320408163265306
1.3338775510204082
1.4357142857142857
1.5375510204081633
1.6393877551020408
1.7412244897959184
1.843061224489796
1.9448979591836735
2.046734693877551
2.1485714285714286
2.250408163265306
2.3522448979591832
2.454081632653061
2.555918367346939
2.657755102040816
2.7595918367346934
2.861428571428571
2.963265306122449
3.0651020408163263
3.1669387755102036
3.2687755102040814
3.370612244897959
3.4724489795918365
3.574285714285714
3.6761224489795916
3.7779591836734694
3.8797959183673467
3.981632653061224
4.083469387755102
4.18530612244898
4.287142857142857
4.388979591836734
4.490816326530612


In [49]:
# Ploteamos
rho_range = dict(sorted(rho_range.items()))
x_axis = list(rho_range.keys())
values = list(rho_range.items())
size = len(values[0][1])

fig = go.Figure()
for k in range(0,size):
    fig.add_trace(go.Scatter(
        x=x_axis,
        y=[values[j][1][k] for j in range(0,num)]
    ))
fig.update_layout(xaxis_title='G',
                   yaxis_title='Rho2')
fig.show()

In [50]:
values[10][1]

array([0.0085655 , 0.00856557, 0.01164046, 0.01858105, 0.01858106,
       0.01858108, 0.01858111, 0.0390528 , 0.03905284, 0.07288291,
       0.09460908, 0.09460909, 0.09460913, 0.09460914, 0.18249387,
       0.18249415, 0.18249425, 0.18249427, 0.65669745, 0.6566978 ,
       3.324108  ], dtype=float32)

In [5]:
m = 4
d = 2
# Creo las bases para no tener que recrearlas luego
basis = fixed_basis(m, 2*d)
basis_m1 = fixed_basis(m-1, 2*d)
basis_m2 = fixed_basis(m-2, 2*d)
print(basis.base)
e = 1
eps = 0
e0 = np.zeros(2*d)
for k in range(0, d):
    r = random.random() * eps
    e0[2*k] = k*e+r
    e0[2*k+1] = k*e+r

h = hamiltonian(2, basis, basis_m1, basis_m2)
ha = np.array(h.todense())
ha.astype(np.float32).tofile("myData.dat")
basis.size

[[0 0 0 4]
 [0 0 1 3]
 [0 0 2 2]
 [0 0 3 1]
 [0 0 4 0]
 [0 1 0 3]
 [0 1 1 2]
 [0 1 2 1]
 [0 1 3 0]
 [0 2 0 2]
 [0 2 1 1]
 [0 2 2 0]
 [0 3 0 1]
 [0 3 1 0]
 [0 4 0 0]
 [1 0 0 3]
 [1 0 1 2]
 [1 0 2 1]
 [1 0 3 0]
 [1 1 0 2]
 [1 1 1 1]
 [1 1 2 0]
 [1 2 0 1]
 [1 2 1 0]
 [1 3 0 0]
 [2 0 0 2]
 [2 0 1 1]
 [2 0 2 0]
 [2 1 0 1]
 [2 1 1 0]
 [2 2 0 0]
 [3 0 0 1]
 [3 0 1 0]
 [3 1 0 0]
 [4 0 0 0]]


35

In [6]:
h = hamiltonian(0.57, basis, basis_m1, basis_m2)
sol = linalg.eigsh(h,which='SA',k=19,return_eigenvectors=True)
eig = sol[0].real
eigv = sol[1]
argmin = np.argmin(sol[0].real)
fund = sol[1][:,argmin]
fund = fund.real / np.linalg.norm(fund)
rho = rho_m(basis, 1, fund).todense()
print(linalg_d.eigvals(rho).real)
eig

[0.12368441 0.12368453 1.8763157  1.8763165 ]


array([-2.6804726e+00, -1.9946163e+00, -1.9946148e+00, -6.5644348e-01,
       -6.5644300e-01, -6.5644288e-01, -6.5644217e-01, -2.3859553e-07,
        7.1170712e-07,  3.7932914e-01,  7.8315294e-01,  7.8315294e-01,
        9.9999905e-01,  9.9999928e-01,  1.0000001e+00,  1.0000002e+00,
        1.7146156e+00,  1.7146163e+00,  2.0000000e+00], dtype=float32)

In [18]:
rho1 = rho_1(basis, fund).todense()
print(np.trace(rho1))
print(linalg_d.eigvals(rho1).real)

3.9999998
[1.8763163  1.876315   0.12368434 0.12368418]


In [13]:
rho2 = rho_2(basis, fund).todense()
print(np.trace(rho2))
print(linalg_d.eigvals(rho2).real)
np.set_printoptions(suppress=True)


6.0
[0.00731809 3.7699203  0.00731803 0.01199096 0.88363785 0.10904462
 0.88363725 0.10904456 0.10904457 0.10904454]


In [28]:
g = gamma(basis, 2, fund).todense()
rho = np.dot(g,np.transpose(g))
print(np.trace(rho))
fund

0 2
[0 0 0 2] [0 0 2 0] 0.08554583787918091
1 1
[0 0 1 1] [0 0 1 1] 0.17109167575836184
1 8
[0 0 1 1] [1 1 0 0] 0.3302189111709595
2 0
[0 0 2 0] [0 0 0 2] 0.08554583787918091
3 7
[0 1 0 1] [1 0 1 0] 0.3302189111709595
4 6
[0 1 1 0] [1 0 0 1] 0.3302189111709595
5 9
[0 2 0 0] [2 0 0 0] 0.9400199055671692
6 4
[1 0 0 1] [0 1 1 0] 0.3302189111709595
7 3
[1 0 1 0] [0 1 0 1] 0.3302189111709595
8 1
[1 1 0 0] [0 0 1 1] 0.3302189111709595
8 8
[1 1 0 0] [1 1 0 0] 1.8800398111343388
9 5
[2 0 0 0] [0 2 0 0] 0.9400199055671692
6.0000005


array([ 0.00000019, -0.00000003,  0.08554584, -0.00000006,  0.00000006,
        0.00000007, -0.00000002, -0.00000005,  0.00000001, -0.        ,
       -0.00000008,  0.00000001,  0.00000005,  0.00000005, -0.        ,
        0.00000008,  0.00000007, -0.00000007,  0.00000002, -0.00000004,
        0.3302189 ,  0.00000004, -0.00000001, -0.00000015, -0.00000048,
        0.00000002,  0.00000011, -0.00000001,  0.00000016, -0.00000014,
        0.9400199 , -0.00000003,  0.00000011,  0.0000002 , -0.00000003],
      dtype=float32)

In [8]:
import time
t = time.time()
r1 = rho_m(basis, 2, fund)
to = time.time()
print(to-t)
t = time.time()
r1 = rho_2(basis, fund)
to = time.time()
print(to-t)

0.030218124389648438
0.31302523612976074
